In [134]:
from graphdatascience import GraphDataScience
import pandas as pd
from py2neo import Node, Graph, Relationship,NodeMatcher
import math
from neo4j import GraphDatabase

In [135]:
df = pd.read_csv("../FinalDataSetCreation/FinalDataset_with_all.csv")
neo_df = df.drop(['url', 'artist_url', 'media_long','dimension','bid','full_bio'],axis = 1)
neo_df.to_csv("neo4j_data.csv")
#neo_df = neo_df[:30]
neo_df['color_group1'] = neo_df['color_group1'].astype('Int64')
neo_df['color_group2'] = neo_df['color_group2'].astype('Int64')
neo_df['color_group3'] = neo_df['color_group3'].astype('Int64')
neo_df['color_group4'] = neo_df['color_group4'].astype('Int64')

In [136]:
"""df = pd.read_csv("../5_InstagramFinder/FinalDataSet.csv")
artist_df = df[['gallery_name']]
len(artist_df.drop_duplicates())
"""

'df = pd.read_csv("../5_InstagramFinder/FinalDataSet.csv")\nartist_df = df[[\'gallery_name\']]\nlen(artist_df.drop_duplicates())\n'

In [137]:
#gqm: for further location (location of gallery) scraping
df[['id','gallery_url']]

,id,gallery_url
0,painting15,https://www.artsy.net/partner/corridor-contemp...
1,prints369,https://www.artsy.net/partner/pinto-gallery
2,prints826,https://www.artsy.net/partner/modclair
3,photography133,https://www.artsy.net/auction/heritage-urban-a...
4,painting37,https://www.artsy.net/partner/maddox-gallery
...,...,...
2474,design1182,https://www.artsy.net/partner/fortes-daloia-an...
2475,design1269,https://www.artsy.net/partner/paradisoterrestre
2476,design1278,https://www.artsy.net/partner/lions-gallery
2477,design1440,https://www.artsy.net/partner/alpha-137-gallery


In [138]:
neo_df.columns

Index(['Unnamed: 0', 'id', 'media', 'artist_name', 'work_name', 'work_year',
       'gallery_name', 'gallery_url', 'price', 'img_url', 'artist_website',
       'artist_ins', 'nationality', 'education', 'location', 'color1',
       'color2', 'color3', 'color4', 'color_group1', 'color_group2',
       'color_group3', 'color_group4', 'class', 'artist_style'],
      dtype='object')

In [139]:
#add color node
color_df = neo_df[['id','color_group1','color_group2','color_group3','color_group4']]
color_list = list(range(1000))
color_dict = {}
for i in range(len(color_df)):
    if not pd.isna(list(color_df.iloc[i])[1]):
        color_dict[list(color_df.iloc[i])[0]] = list(color_df.iloc[i])[1:]
print(len(color_dict))


1990


In [140]:
#add artist node
artist_df = neo_df[['artist_name','artist_website','artist_ins','nationality','education','location','artist_style']]
artist_df.fillna('', inplace=True)
artist_list = []
for i in range(len(artist_df)):
    artist_list.append(list(artist_df.iloc[i]))
temp= []
[temp.append(i) for i in artist_list if not i in temp]
artist_list = temp


/var/folders/2p/2p_g91q95gj9px5nbxvkbw3h0000gn/T/ipykernel_29681/920841734.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  artist_df.fillna('', inplace=True)


In [141]:
#add work node
work_df = neo_df[['id','media','artist_name','work_name','work_year','price']]
work_list = []
for i in range(len(work_df)):
    work_list.append(list(work_df.iloc[i]))
temp= []
[temp.append(i) for i in work_list if not i in temp]
work_list = temp

In [142]:
#add gallery node
gallery_df = neo_df[['gallery_name', "gallery_url",'artist_name','id']]
gallery_list = []
for i in range(len(gallery_df)):
    gallery_list.append(list(gallery_df.iloc[i]))
temp= []
[temp.append(i) for i in gallery_list if not i in temp]
gallery_list = temp

In [143]:
#add stype node:
style_df = neo_df[['id','class']]
style_list = []
for i in range(len(style_df)):
    if not pd.isna(list(style_df.iloc[i])[1]):
        style_list.append(list(style_df.iloc[i]))
temp= []
[temp.append(i) for i in style_list if not i in temp]
style_list = temp



In [144]:
# Aura queries use an encrypted connection using the "neo4j+s" URI scheme
uri = "neo4j+s://39d587d1.databases.neo4j.io"
user = "neo4j"
password = "SwH_uBibcZOdqS-aH2dg6n2qtfXMbfCaVozFwEoy3H4"
graph = Graph(uri, auth=(user, password), routing=True)

In [145]:
for i in artist_list:
    print(i)
    break

['Hebru Brantley', 'www.hebrubrantley.com', 'https://www.instagram.com/hebrubrantley/?hl=en', 'Japanese', 'Clark Atlanta University', 'Chicago', 'post-minimalism']


In [146]:
artist_nodes = {}
for i in artist_list:
    name = i[0]
    web = i[1]
    ins = i[2]
    nationality = i[3]
    education = i[4]
    location = i[5]
    artist_nodes[name] = Node("Artist", name = name)
    if web:
        artist_nodes[name]['website'] = web
    if ins:
        artist_nodes[name]['ins'] = ins
    if nationality:
        artist_nodes[name]['nationality'] = nationality
    if location:
        artist_nodes[name]['location'] = location

In [147]:
work_nodes = {}
artist_work_rel = {}
for i in work_list:
    work_id = i[0]
    media = i[1]
    artist = i[2]
    name = i[3]
    year = int(i[4])
    price = int(i[5])
    work_nodes[work_id] = Node("Artork", id = work_id, media = media, name = name, year = year, price = price)
    #Relationship(LillyW, "DIRECTED", TheMatrix)
    artist_work_rel[artist+work_id] = Relationship(artist_nodes[artist], "CREATED", work_nodes[work_id])


In [148]:
#tx = graph.begin()

#graph.commit(tx)

In [149]:
gallery_nodes = {}
artist_gallery_rel = {}
gallery_work_rel = {}
already = set()
for i in gallery_list:
    gallery_name = i[0]
    gallery_website = i[1]
    artist = i[2]
    work = i[3]
    if gallery_name not in already:
        already.add(gallery_name)
        gallery_nodes[gallery_name] = Node("Gallery", name = gallery_name, website = gallery_website)
    #Relationship(LillyW, "DIRECTED", TheMatrix)
    if artist+gallery_name not in artist_gallery_rel:
        artist_gallery_rel[artist+gallery_name] = Relationship(artist_nodes[artist], "HASWORK", gallery_nodes[gallery_name])
    gallery_work_rel[work+gallery_name] = Relationship(work_nodes[work], "DISPLAY", gallery_nodes[gallery_name])

In [150]:
style_nodes = {}
style_work_rel = {}

style_types = set([x[1] for x in style_list])
for i in style_types:
    style_nodes[i] = Node("Style", name = i)

for i in style_list:
   
    style_name = i[1]
    work = i[0]
    #print(style_name, work)
    style_work_rel[style_name+work] = Relationship(work_nodes[work], "HASSTYLE", style_nodes[style_name])

In [151]:
color_nodes = {}
work_color_rel = {}

for i in color_list:
    color_nodes[i] = Node("Color", name = i)

for i in color_dict.keys():
   
    work = i
    colors = color_dict[work]
    
    for col in colors:
        if col == -1:
            pass
        else:
            work_color_rel[work+str(col)] = Relationship(work_nodes[work], "HASCOLOR", color_nodes[col])


In [152]:
same_gallery_rel = []

gallery_artist = {}
for i in gallery_list:
    gallery_name = i[0]
    artist = i[2]
    if gallery_name in gallery_artist:
        gallery_artist[gallery_name].append(artist_nodes[artist])
    else:
        gallery_artist[gallery_name] = [artist_nodes[artist]]

for test_list in gallery_artist.values():
    temp = [[a, b] for idx, a in enumerate(test_list) for b in test_list[idx + 1:]]
    for [a,b] in temp:
        same_gallery_rel.append(Relationship(a, "SAMEGALLERY", b))


In [153]:
artist_list[0]

['Hebru Brantley',
 'www.hebrubrantley.com',
 'https://www.instagram.com/hebrubrantley/?hl=en',
 'Japanese',
 'Clark Atlanta University',
 'Chicago',
 'post-minimalism']

In [154]:
artist_list[0]
artist_style_rel = []
for i in artist_list:
    artist = i[0]
    style = i[-1]
    if style != 'no distinct style':
        artist_style_rel.append(Relationship(artist_nodes[artist],"HASSTYLE",style_nodes[style]))


In [155]:
tx = graph.begin()
for i in artist_nodes.values():
    tx.create(i)
for i in work_nodes.values():
    tx.create(i)
for i in gallery_nodes.values():
    tx.create(i)
for i in style_nodes.values():
    tx.create(i)
for i in color_nodes.values():
    tx.create(i)
for i in artist_work_rel.values():
    tx.create(i)
for i in artist_gallery_rel.values():
    tx.create(i)
for i in gallery_work_rel.values():
    tx.create(i)
for i in same_gallery_rel:
    tx.create(i)
for i in style_work_rel.values():
    tx.create(i)
for i in artist_style_rel:
    tx.create(i)
for i in work_color_rel:
    tx.create(i)
    
graph.commit(tx)

In [67]:
"""tx = graph.begin()
for i in style_work_rel.values():
    tx.create(i)
for i in work_nodes.values():
    tx.create(i)
for i in style_nodes.values():
    tx.create(i)
graph.commit(tx)"""

'tx = graph.begin()\nfor i in style_work_rel.values():\n    tx.create(i)\nfor i in work_nodes.values():\n    tx.create(i)\nfor i in style_nodes.values():\n    tx.create(i)\ngraph.commit(tx)'

In [68]:
"""MATCH (n:MyImageNode)
SET n.image_url = "http://myimage.png"
RETURN n"""

'MATCH (n:MyImageNode)\nSET n.image_url = "http://myimage.png"\nRETURN n'

In [70]:
#get img list
img_df = neo_df[['id','img_url']]
img_list = []
for i in range(len(img_df)):
    img_list.append(list(img_df.iloc[i]))
temp= []
[temp.append(i) for i in img_list if not i in temp]
img_list = temp

In [71]:
query_list = []
for i in img_list:
    query = 'MATCH (n:Artork) WHERE n.id = "{}" SET n.image_url = "{}" RETURN n'.format(i[0],i[1])
    query_list.append(query)

In [74]:
driver = GraphDatabase.driver("neo4j+s://39d587d1.databases.neo4j.io", auth=("neo4j", "SwH_uBibcZOdqS-aH2dg6n2qtfXMbfCaVozFwEoy3H4"))
with driver.session() as session:
    for query in query_list:
        results = session.run(query).data()
        print(results)

[{'n': {'year': 2020, 'image_url': 'https://d7hftxdivxxvm.cloudfront.net?resize_to=fit&width=600&height=800&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2F03VidwQCDEq-r0T3Ck3CcA%2Fnormalized.jpg', 'price': 125000, 'name': 'Death Of The Black Fairy, The Great Debate Pt. 5', 'media': 'painting', 'id': 'painting15'}}]
[{'n': {'year': 2022, 'image_url': 'https://d7hftxdivxxvm.cloudfront.net?resize_to=fit&width=639&height=800&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FC8CkTBthzSHZY08MLF0zFA%2Fnormalized.jpg', 'price': 17500, 'name': 'Maestro', 'id': 'painting37', 'media': 'painting'}}]
[{'n': {'year': 2017, 'image_url': 'https://d7hftxdivxxvm.cloudfront.net?resize_to=fit&width=800&height=768&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2F6iXBZQgdBLRzr7OtGn0zOA%2Fnormalized.jpg', 'price': 5000, 'name': 'La cage où le brouillard traverse l’esprit', 'media': 'painting', 'id': 'painting41'}}]
[{'n': {'year': 2022, 'image_url': 'https://d7hftxdivxxvm